# Генерация датасета для OBB

In [166]:
import os
from PIL import Image, ImageDraw
import numpy as np
import random
from shapely.geometry import Polygon
import glob
import copy
import cv2

In [167]:
SUBDIR_FILES = os.path.join('data', 'generation', 'data')
DATA_CROP_PATH = os.path.join(os.getcwd(), SUBDIR_FILES)
DATA_GENERATE_PATH = os.path.join(os.getcwd(), 'data', 'retina-yolo')

In [168]:
types_del = ('*.txt', '*.jpg', '*.jpeg') # the tuple of file types
files_del = []
for files in types_del:
    search_path = DATA_GENERATE_PATH + '\\**\\' + files
    print(search_path)
    files_del.extend(glob.glob(search_path, recursive=True))

c:\Work\ML\retina-obb-gen-data\data\retina-yolo\**\*.txt
c:\Work\ML\retina-obb-gen-data\data\retina-yolo\**\*.jpg
c:\Work\ML\retina-obb-gen-data\data\retina-yolo\**\*.jpeg


In [169]:
for p in files_del:
    os.remove(p)

In [170]:
types = ('*.png', '*.jpg', '*.jpeg') # the tuple of file types
files_grabbed = []
for files in types:
    search_path = DATA_CROP_PATH + '\\**\\' + files
    print(search_path)
    files_grabbed.extend(glob.glob(search_path, recursive=True))

c:\Work\ML\retina-obb-gen-data\data\generation\data\**\*.png
c:\Work\ML\retina-obb-gen-data\data\generation\data\**\*.jpg
c:\Work\ML\retina-obb-gen-data\data\generation\data\**\*.jpeg


Перемешиваем

In [171]:
random.shuffle(files_grabbed)

In [172]:
import itertools

class Rectangle:
    def intersection(self, other):
        a, b = self, other
        x1 = max(min(a.x1, a.x2), min(b.x1, b.x2))
        y1 = max(min(a.y1, a.y2), min(b.y1, b.y2))
        x2 = min(max(a.x1, a.x2), max(b.x1, b.x2))
        y2 = min(max(a.y1, a.y2), max(b.y1, b.y2))
        if x1<x2 and y1<y2:
            return type(self)(x1, y1, x2, y2)
    __and__ = intersection

    def difference(self, other):
        inter = self&other
        if not inter:
            yield self
            return
        xs = {self.x1, self.x2}
        ys = {self.y1, self.y2}
        if self.x1<other.x1<self.x2: xs.add(other.x1)
        if self.x1<other.x2<self.x2: xs.add(other.x2)
        if self.y1<other.y1<self.y2: ys.add(other.y1)
        if self.y1<other.y2<self.y2: ys.add(other.y2)
        for (x1, x2), (y1, y2) in itertools.product(
            pairwise(sorted(xs)), pairwise(sorted(ys))
        ):
            rect = type(self)(x1, y1, x2, y2)
            if rect!=inter:
                yield rect
    __sub__ = difference

    def __init__(self, x1, y1, x2, y2):
        if x1>x2 or y1>y2:
            raise ValueError("Coordinates are invalid")
        self.x1, self.y1, self.x2, self.y2 = x1, y1, x2, y2

    def __iter__(self):
        yield self.x1
        yield self.y1
        yield self.x2
        yield self.y2

    def __eq__(self, other):
        return isinstance(other, Rectangle) and tuple(self)==tuple(other)
    def __ne__(self, other):
        return not (self==other)

    def __repr__(self):
        return type(self).__name__+repr(tuple(self))

In [173]:
def check_intersection(x1_tl, y1_tl, x1_br, y1_br, x2_tl, y2_tl, x2_br, y2_br):
    r1 = Rectangle(x1_tl-5, y1_tl-5, x1_br+5, y1_br+5)
    r2 = Rectangle(x2_tl-5, y2_tl-5, x2_br+5, y2_br+5)
    intersection = r1&r2
    return intersection

In [174]:
def generate_image(image_pathes, img_save_path, is_draw=False):
    insert_images = [] # массив объектов с координатам вставленных картинок
    # на случай если не судьба 
    limit_try = 300
    
    basewidth = 650+random.randint(-50, 50)
    images_list = image_pathes
    canvas_width = 800
    canvas_height = 800
    r_noise = random.randint(-10,10)
    g_noise = random.randint(-10,10)
    b_noise = random.randint(-10,10)
    img_new = Image.new('RGB', (canvas_width, canvas_height), color = (50 + r_noise, 50 + g_noise, 50 + b_noise))
    if random.randint(-5, 5) > 0:
        img_new = Image.new('RGB', (canvas_width, canvas_height), color = (245 + r_noise, 245 + g_noise, 245 + b_noise))
    draw = ImageDraw.Draw(img_new)
    # Выбираем случайный цвет
    R, G, B = random.randint(33,53), random.randint(33,53), random.randint(33,53),

    # Генерим случайное число кругов как шум на картинке
    cmax = random.randint(0,800)
    for _ in range(cmax):
        r = R + random.randint(-20,20)
        g = G + random.randint(-20,20)
        b = B + random.randint(-20,20)
        diam = random.randint(15,70)
        x, y = random.randint(0,canvas_width), random.randint(0, canvas_height)
        draw.ellipse([x,y,x+diam,y+diam], fill=(r,g,b))
    
    # в цикле пробуем поставить изображение
    for img_index, image_retina_filepath in enumerate(images_list):
        iii = 0
        retina_path = image_retina_filepath
        retina_image = Image.open(retina_path) # парсим картинку
        # ресайзим ее
        w_resize = random.randint(0, 150)
        h_resize = random.randint(0, 200)
        retina_image = retina_image.resize([510 + w_resize, 250 + h_resize], Image.Resampling.LANCZOS)
        # получаем ее новую высоту и ширину
        img_width, img_height = retina_image.size
        retina_class = 'retina_horizontal' if not retina_path.endswith('_1.jpg') else 'retina_square'

        while True:
            iii = iii+1
            
            if iii > limit_try:
                return False
            
            # получаем случайные координаты
            x_top_left = random.randint(10, canvas_width-img_width-10) # вычитаем от бока так как не залезет все равно
            y_top_left = random.randint(10, canvas_height-img_height-10) # аналогично вычитаем
            
            # проверяем что не попали не в одну картинку
            is_exists_intersect = False
            for crop_image_ins in insert_images:
                intersect = check_intersection(crop_image_ins['x1'], crop_image_ins['y1'], crop_image_ins['x3'], crop_image_ins['y3'],x_top_left, y_top_left, x_top_left+img_width, y_top_left+img_height)
                if intersect is not None:
                    is_exists_intersect = True
                    break
                
            if is_exists_intersect == False:
                image_ins_obj = {
                    'class': retina_class,
                    'x1': x_top_left,
                    'y1': y_top_left,
                    'x2': x_top_left+img_width,
                    'y2': y_top_left,
                    'x3': x_top_left+img_width,
                    'y3': y_top_left+img_height,
                    'x4': x_top_left,
                    'y4': y_top_left+img_height,
                    'iii': iii,
                }
                image_ins_obj['x1_n'] = image_ins_obj['x1'] / canvas_width
                image_ins_obj['y1_n'] = image_ins_obj['y1'] / canvas_width
                image_ins_obj['x2_n'] = image_ins_obj['x2'] / canvas_width
                image_ins_obj['y2_n'] = image_ins_obj['y2'] / canvas_width
                image_ins_obj['x3_n'] = image_ins_obj['x3'] / canvas_width
                image_ins_obj['y3_n'] = image_ins_obj['y3'] / canvas_width
                image_ins_obj['x4_n'] = image_ins_obj['x4'] / canvas_width
                image_ins_obj['y4_n'] = image_ins_obj['y4'] / canvas_width
                insert_images.append(image_ins_obj)
                img_new.paste(retina_image, (x_top_left, y_top_left))
                break

    open_cv_image = np.array(img_new)
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    center = (open_cv_image.shape[1] // 2, open_cv_image.shape[0] // 2)
    
    # Пробуем изображение перевернуть
    iii = 0
    check_coord = lambda x: x > 0 and x < canvas_width
    while True:
        iii += 1
        img_try_rotate = open_cv_image.copy() 
        insert_images_try_rotate = [copy.copy(d) for d in insert_images]
        if iii > limit_try:
            # Просто break значит переворачивать не будем
            break

        # Пробуем перевернуть
        angle = random.randint(-15, 15)
        angle_case = random.randint(1, 5)
        if angle_case == 3:
            angle = random.randint(75, 105)

        if angle_case == 4:
            angle = random.randint(165, 195)

        if angle_case == 5:
            angle = random.randint(255, 285)


        # create the rotation matrix which is required to rotate images and points (everything as a numpy array)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated_image = cv2.warpAffine(img_try_rotate, rotation_matrix, img_try_rotate.shape[:2])


        # Теперь перезаписываем координаты:
        for ind, item in enumerate(insert_images_try_rotate):
            # x1 and y1
            point_to_track = (item['x1'], item['y1'])
            point_after_rotation = cv2.transform(np.array([[point_to_track]]), rotation_matrix)[0][0]
            insert_images_try_rotate[ind]['x1'] = point_after_rotation[0]
            insert_images_try_rotate[ind]['y1'] = point_after_rotation[1]
            # x2 and y2
            point_to_track = (item['x2'], item['y2'])
            point_after_rotation = cv2.transform(np.array([[point_to_track]]), rotation_matrix)[0][0]
            insert_images_try_rotate[ind]['x2'] = point_after_rotation[0]
            insert_images_try_rotate[ind]['y2'] = point_after_rotation[1]
            # x3 and y3
            point_to_track = (item['x3'], item['y3'])
            point_after_rotation = cv2.transform(np.array([[point_to_track]]), rotation_matrix)[0][0]
            insert_images_try_rotate[ind]['x3'] = point_after_rotation[0]
            insert_images_try_rotate[ind]['y3'] = point_after_rotation[1]
            # x4 and y4
            point_to_track = (item['x4'], item['y4'])
            point_after_rotation = cv2.transform(np.array([[point_to_track]]), rotation_matrix)[0][0]
            insert_images_try_rotate[ind]['x4'] = point_after_rotation[0]
            insert_images_try_rotate[ind]['y4'] = point_after_rotation[1]

        # чек что координаты все не выпали
        count_check = 0
        for item in insert_images_try_rotate:
            if check_coord(item['x1']) and check_coord(item['y1']) and \
                check_coord(item['x2']) and check_coord(item['y2']) and \
                check_coord(item['x3']) and check_coord(item['y3']) and \
                check_coord(item['x4']) and check_coord(item['y4']):
                count_check += 1
            else:
                continue
        
        # Если все ок
        if count_check == len(insert_images_try_rotate):
            color_converted = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2RGB)
            img_new = Image.fromarray(color_converted)
            insert_images = insert_images_try_rotate
            for ind, image_ins_obj in enumerate(insert_images):
                insert_images[ind]['x1_n'] = image_ins_obj['x1'] / canvas_width
                insert_images[ind]['y1_n'] = image_ins_obj['y1'] / canvas_width
                insert_images[ind]['x2_n'] = image_ins_obj['x2'] / canvas_width
                insert_images[ind]['y2_n'] = image_ins_obj['y2'] / canvas_width
                insert_images[ind]['x3_n'] = image_ins_obj['x3'] / canvas_width
                insert_images[ind]['y3_n'] = image_ins_obj['y3'] / canvas_width
                insert_images[ind]['x4_n'] = image_ins_obj['x4'] / canvas_width
                insert_images[ind]['y4_n'] = image_ins_obj['y4'] / canvas_width
            break

    if is_draw:
        # Если нужно проверить отрисовать линии
        img_new_draw = ImageDraw.Draw(img_new)
        for ins_bb in insert_images:
            img_new_draw.line([ins_bb['x1'], ins_bb['y1'], ins_bb['x2'], ins_bb['y2']], fill ="blue", width = 5)
            img_new_draw.line([ins_bb['x2'], ins_bb['y2'], ins_bb['x3'], ins_bb['y3']], fill ="green", width = 5)
            img_new_draw.line([ins_bb['x3'], ins_bb['y3'], ins_bb['x4'], ins_bb['y4']], fill ="blue", width = 5)
            img_new_draw.line([ins_bb['x4'], ins_bb['y4'], ins_bb['x1'], ins_bb['y1']], fill ="green", width = 5)

    img_new.save(img_save_path)
    return insert_images

In [175]:
files_grabbed

['c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\103150research\\retina_map_3.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\114067research\\retina_map_2.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\132360research\\retina_map_1.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\60761research\\retina_map_2.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\98944research\\retina_map_3.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\56639research\\retina_map_1.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\101721research\\retina_map_1.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\123383research\\retina_map_1.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\data\\generation\\data\\old\\crop\\155119research\\retina_map_3.jpg',
 'c:\\Work\\ML\\retina-obb-gen-data\\dat

In [176]:
files_grabbed_filter = [path for path in files_grabbed if not path.endswith('_1.jpg')]

In [177]:
len(files_grabbed_filter)

19629

In [178]:
COUNT_IMG_TRAIN = 200
COUNT_IMG_VAL = 56
COUNT_IMG_TEST = 200

In [179]:
img_pathes_train = list(files_grabbed_filter[:COUNT_IMG_TRAIN])
img_pathes_val = list(files_grabbed_filter[COUNT_IMG_TRAIN:COUNT_IMG_TRAIN+COUNT_IMG_VAL])
img_pathes_test = list(files_grabbed_filter[COUNT_IMG_TRAIN+COUNT_IMG_VAL:COUNT_IMG_TRAIN+COUNT_IMG_VAL+COUNT_IMG_TEST])

## Генерация данных для обучения

In [180]:
IMG_ID = 0

In [181]:
LABEL_ENCODER = {'retina_horizontal': 0, 'retina_square': 1}

In [182]:
droped_img = 0
while len(img_pathes_train) > 0:
    IMG_ID += 1

    if(droped_img%50==0):
        print(f'processing {droped_img/COUNT_IMG_TRAIN*100}%')

    file_save_gen_path = os.path.join(DATA_GENERATE_PATH, 'images', 'train', f'{IMG_ID}.jpg')
    save_txt_path = os.path.join(DATA_GENERATE_PATH, 'labels', 'train', f'{IMG_ID}.txt')
    save_txt_orig_path = os.path.join(DATA_GENERATE_PATH, 'labels', 'train_original', f'{IMG_ID}.txt')
    # генерим сколько резов будет на изображении
    count_bb_on_image = random.randint(1, 2)
    list_images = []
    # Собираем резы
    for i in range(count_bb_on_image):
        droped_img += 1
        if len(img_pathes_train) > 0:
            list_images.append(img_pathes_train.pop())

    generate_image_data = generate_image(list_images, file_save_gen_path)
    if generate_image_data == False:
        continue

    with open(save_txt_orig_path, "w") as fp:
        arr = [f'{item['class']} {item['x1']} {item['y1']} {item['x2']} {item['y2']} {item['x3']} {item['y3']} {item['x4']} {item['y4']}' for item in generate_image_data]
        fp.writelines('\n'.join(arr))

    with open(save_txt_path, "w") as fp:
        arr = [f'{LABEL_ENCODER[item['class']]} {item['x1_n']} {item['y1_n']} {item['x2_n']} {item['y2_n']} {item['x3_n']} {item['y3_n']} {item['x4_n']} {item['y4_n']}' for item in generate_image_data]
        fp.writelines('\n'.join(arr))

processing 0.0%
processing 25.0%
processing 50.0%
processing 75.0%


## Генерация данных для валидации

In [183]:
droped_img = 0
while len(img_pathes_val) > 0:
    IMG_ID += 1

    if(droped_img%50==0):
        print(f'processing {droped_img/COUNT_IMG_VAL*100}%')

    file_save_gen_path = os.path.join(DATA_GENERATE_PATH, 'images', 'val', f'{IMG_ID}.jpg')
    save_txt_path = os.path.join(DATA_GENERATE_PATH, 'labels', 'val', f'{IMG_ID}.txt')
    save_txt_orig_path = os.path.join(DATA_GENERATE_PATH, 'labels', 'val_original', f'{IMG_ID}.txt')
    # генерим сколько резов будет на изображении
    count_bb_on_image = random.randint(1, 2)
    list_images = []
    # Собираем резы
    for i in range(count_bb_on_image):
        droped_img += 1
        if len(img_pathes_val) > 0:
            list_images.append(img_pathes_val.pop())

    generate_image_data = generate_image(list_images, file_save_gen_path)
    if generate_image_data == False:
        continue

    with open(save_txt_orig_path, "w") as fp:
        arr = [f'{item['class']} {item['x1']} {item['y1']} {item['x2']} {item['y2']} {item['x3']} {item['y3']} {item['x4']} {item['y4']}' for item in generate_image_data]
        fp.writelines('\n'.join(arr))

    with open(save_txt_path, "w") as fp:
        arr = [f'{LABEL_ENCODER[item['class']]} {item['x1_n']} {item['y1_n']} {item['x2_n']} {item['y2_n']} {item['x3_n']} {item['y3_n']} {item['x4_n']} {item['y4_n']}' for item in generate_image_data]
        fp.writelines('\n'.join(arr))

processing 0.0%


## Генерация данных для теста

In [95]:
droped_img = 0
while len(img_pathes_test) > 0:
    IMG_ID += 1

    if(droped_img%50==0):
        print(f'processing {droped_img/COUNT_IMG_TEST*100}%')

    file_save_gen_path = os.path.join(DATA_GENERATE_PATH, 'images', 'test', f'{IMG_ID}.jpg')
    save_txt_path = os.path.join(DATA_GENERATE_PATH, 'labels', 'test', f'{IMG_ID}.txt')
    save_txt_orig_path = os.path.join(DATA_GENERATE_PATH, 'labels', 'test_original', f'{IMG_ID}.txt')
    # генерим сколько резов будет на изображении
    count_bb_on_image = random.randint(1, 2)
    list_images = []
    # Собираем резы
    for i in range(count_bb_on_image):
        droped_img += 1
        if len(img_pathes_test) > 0:
            list_images.append(img_pathes_test.pop())

    generate_image_data = generate_image(list_images, file_save_gen_path)
    if generate_image_data == False:
        continue

    with open(save_txt_orig_path, "w") as fp:
        arr = [f'{item['class']} {item['x1']} {item['y1']} {item['x2']} {item['y2']} {item['x3']} {item['y3']} {item['x4']} {item['y4']}' for item in generate_image_data]
        fp.writelines('\n'.join(arr))

    with open(save_txt_path, "w") as fp:
        arr = [f'{LABEL_ENCODER[item['class']]} {item['x1_n']} {item['y1_n']} {item['x2_n']} {item['y2_n']} {item['x3_n']} {item['y3_n']} {item['x4_n']} {item['y4_n']}' for item in generate_image_data]
        fp.writelines('\n'.join(arr))

processing 0.0%
processing 2.5%
processing 5.0%
processing 7.5%
processing 10.0%
processing 15.0%
processing 17.5%
processing 20.0%
processing 22.5%
processing 25.0%
processing 27.500000000000004%
processing 32.5%
processing 35.0%
processing 37.5%
processing 40.0%
processing 42.5%
processing 47.5%
processing 50.0%
processing 52.5%
processing 55.00000000000001%
processing 60.0%
processing 62.5%
processing 65.0%
processing 67.5%
processing 70.0%
processing 80.0%
processing 85.0%
processing 87.5%
processing 90.0%
processing 92.5%
processing 97.5%
